In [1]:
# =========================
# SILCO Fire & Security — Synthetic Data Lab (Colab-ready)
# Author: Zach (FourTwenty Analytics) | Module: The Protector
# =========================

# --- Setup
!pip -q install pandas numpy scipy statsmodels scikit-learn faker pyarrow

import numpy as np
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os, zipfile, io
rng = np.random.default_rng(42)
fake = Faker()

# --- Parameters you can tweak quickly
N_SITES        = 3500        # number of customer locations
YEARS_BACK     = 3           # history window
DRILLS_PER_YR  = (6, 12)     # min/max scheduled drills per site per year
MAINT_PER_YR   = (2, 6)      # scheduled maintenance per site per year

# Incident volume scales with risk, size & compliance gaps
BASE_INCIDENT_RATE = 0.12    # baseline annual incident prob per site
FALSE_ALARM_RATE   = 0.55    # share of incidents that are false alarms

# --- Helpers
def random_choice_weighted(options, weights):
    return rng.choice(options, p=np.array(weights)/np.sum(weights))

def bounded_norm(mu, sigma, low, high, size=None):
    x = rng.normal(mu, sigma, size=size)
    return np.clip(x, low, high)

today = pd.Timestamp.today().normalize()
start_date = today - pd.Timedelta(days=365*YEARS_BACK)

# --- 1) SITES
markets = ["Cincinnati", "Dayton", "Columbus", "Akron", "Cleveland", "Louisville"]
site_types = ["Office", "Warehouse", "Manufacturing", "Retail", "Healthcare", "Education", "Hospitality"]
occupancy_class = ["Light", "Ordinary", "Moderate", "High"]  # NFPA-inspired simplification

site_rows = []
for i in range(1, N_SITES+1):
    installed_year = rng.integers(1995, 2025)
    age = max(0, today.year - installed_year)
    t = random_choice_weighted(site_types, [18,16,14,22,10,10,10])
    occ = random_choice_weighted(occupancy_class, [30,35,25,10])
    sq_ft = int(bounded_norm(45000, 30000, 2000, 250000))
    sprinklers = rng.choice([True, False], p=[0.72, 0.28])
    address = fake.street_address()
    city = rng.choice(markets)
    jurisdiction = f"{city} Fire Dept"
    # Risk score: age, occupancy, sprinklers inverse, size
    risk = (
        0.35*(age/30) +
        0.25*({"Light":0.1,"Ordinary":0.4,"Moderate":0.65,"High":0.9}[occ]) +
        0.25*(0.0 if sprinklers else 0.8) +
        0.15*(min(sq_ft, 200000)/200000)
    )
    risk = float(np.clip(risk, 0, 1))
    service_level = random_choice_weighted(["Gold","Silver","Bronze"], [0.35,0.45,0.20])
    site_rows.append({
        "site_id": i,
        "site_name": f"{t} #{i}",
        "site_type": t,
        "occupancy_class": occ,
        "has_sprinklers": sprinklers,
        "sq_ft": sq_ft,
        "city": city,
        "address": address,
        "ahj_jurisdiction": jurisdiction,
        "installed_year": installed_year,
        "system_age_years": age,
        "service_level": service_level,
        "risk_score": risk
    })

sites = pd.DataFrame(site_rows)

# --- 2) DRILLS (scheduled + outcomes)
drill_rows = []
for _, s in sites.iterrows():
    # higher-risk sites schedule slightly more drills
    per_year = rng.integers(DRILLS_PER_YR[0], DRILLS_PER_YR[1]+1) + int(round(2*s.risk_score))
    total = per_year * YEARS_BACK
    for _ in range(total):
        date = start_date + pd.Timedelta(days=int(rng.integers(0, 365*YEARS_BACK)))
        # Likelihood of completion depends on service level (Gold best)
        lvl = s.service_level
        base_complete = {"Gold":0.90,"Silver":0.82,"Bronze":0.72}[lvl]
        complete = rng.random() < (base_complete - 0.08*(s.risk_score))
        pass_prob = 0.88 - 0.10*s.risk_score + (0.05 if s.has_sprinklers else -0.04)
        passed = (rng.random() < pass_prob) if complete else False
        # Time to evacuate (sec): lower is better; impacted by drills done & risk
        evac_time = bounded_norm(180 - 35*passed + 25*s.risk_score, 25, 60, 600)
        drill_rows.append({
            "drill_id": len(drill_rows)+1,
            "site_id": s.site_id,
            "scheduled_date": date.date(),
            "completed": complete,
            "passed": passed,
            "evac_time_seconds": int(evac_time)
        })

drills = pd.DataFrame(drill_rows)

# --- 3) MAINTENANCE (scheduled events + completion)
maint_rows = []
for _, s in sites.iterrows():
    per_year = rng.integers(MAINT_PER_YR[0], MAINT_PER_YR[1]+1)
    total = per_year * YEARS_BACK
    for _ in range(total):
        date = start_date + pd.Timedelta(days=int(rng.integers(0, 365*YEARS_BACK)))
        # Completion odds vary by service level; missed maint raises incidents
        base_comp = {"Gold":0.93,"Silver":0.86,"Bronze":0.76}[s.service_level]
        completed = rng.random() < (base_comp - 0.10*s.risk_score)
        # Cost varies by size & risk; completed tasks incur cost
        cost = bounded_norm(450 + 0.002*s.sq_ft + 350*s.risk_score, 120, 50, 10000)
        maint_rows.append({
            "maintenance_id": len(maint_rows)+1,
            "site_id": s.site_id,
            "scheduled_date": date.date(),
            "completed": completed,
            "technician": fake.name(),
            "task_type": rng.choice(["Inspection","Service","Repair","Replace"], p=[0.5,0.28,0.17,0.05]),
            "parts_cost": round(cost if completed else 0.0, 2)
        })

maintenance = pd.DataFrame(maint_rows)

# --- 4) INCIDENTS (true incidents + false alarms), with realistic correlations
# Derive compliance features per site/month
drills["month"] = pd.to_datetime(drills["scheduled_date"]).dt.to_period("M").astype(str)
maintenance["month"] = pd.to_datetime(maintenance["scheduled_date"]).dt.to_period("M").astype(str)

drill_agg = (drills.groupby(["site_id","month"])
             .agg(drills_scheduled=("completed","size"),
                  drills_completed=("completed","sum"),
                  drills_passed=("passed","sum"))
             .reset_index())
drill_agg["drill_complete_rate"] = (drill_agg["drills_completed"] / drill_agg["drills_scheduled"]).fillna(0)
drill_agg["drill_pass_rate"] = (drill_agg["drills_passed"] / drill_agg["drills_scheduled"]).fillna(0)

maint_agg = (maintenance.groupby(["site_id","month"])
             .agg(maint_scheduled=("completed","size"),
                  maint_completed=("completed","sum"),
                  maint_spend=("parts_cost","sum"))
             .reset_index())
maint_agg["maint_complete_rate"] = (maint_agg["maint_completed"] / maint_agg["maint_scheduled"]).fillna(0)

# Build month grid for each site
months = pd.period_range(start_date.to_period("M"), today.to_period("M"), freq="M").astype(str)
grid = pd.MultiIndex.from_product([sites["site_id"], months], names=["site_id","month"]).to_frame(index=False)

# Merge compliance features
site_feats = grid.merge(drill_agg, on=["site_id","month"], how="left") \
                 .merge(maint_agg, on=["site_id","month"], how="left") \
                 .merge(sites[["site_id","risk_score","has_sprinklers","service_level","system_age_years","sq_ft"]],
                        on="site_id", how="left") \
                 .fillna({"drills_scheduled":0, "drills_completed":0, "drills_passed":0,
                          "drill_complete_rate":0, "drill_pass_rate":0,
                          "maint_scheduled":0, "maint_completed":0,
                          "maint_spend":0.0, "maint_complete_rate":0.0})

# Monthly incident probability model
def monthly_incident_prob(row):
    # baseline scaled by risk & age
    base = BASE_INCIDENT_RATE * (1 + 0.9*row.risk_score + 0.25*(row.system_age_years/30))
    # mitigations
    mitig = 1 - 0.30*row.maint_complete_rate - 0.25*row.drill_complete_rate - (0.08 if row.has_sprinklers else 0.0)
    # bounds
    return float(np.clip(base*mitig, 0.01, 0.65))

inc_rows = []
incident_id = 1
for _, r in site_feats.iterrows():
    p_inc = monthly_incident_prob(r)
    has_incident = rng.random() < p_inc
    # if there is an incident, maybe multiple in month (rare)
    n_incidents = 0
    if has_incident:
        n_incidents = 1 + (rng.random() < min(0.15 + 0.25*r.risk_score, 0.5))
    for _ in range(int(n_incidents)):
        # severity influenced by drills/maint compliance inversely & risk positively
        sev_mu = 2.2 + 2.0*r.risk_score + 0.6*(1-r.drill_complete_rate) + 0.7*(1-r.maint_complete_rate)
        severity = int(np.clip(round(bounded_norm(sev_mu, 0.9, 1.0, 5.0)), 1, 5))
        itype = "False Alarm" if (rng.random() < FALSE_ALARM_RATE*(1 - 0.15*r.drill_complete_rate)) else "Real Event"
        loss = 0.0
        if itype == "Real Event":
            # property loss grows with severity & size; reduced by sprinklers
            base_loss = 5000*severity + 0.5*r.sq_ft
            if r.has_sprinklers:
                base_loss *= 0.65
            loss = float(np.clip(bounded_norm(base_loss, base_loss*0.35, 250, 250000), 250, 500000))

        # pick a date within the month
        year, month = map(int, r.month.split("-"))
        day = rng.integers(1, 28)
        d = pd.Timestamp(year=year, month=month, day=day).date()
        inc_rows.append({
            "incident_id": incident_id,
            "site_id": int(r.site_id),
            "incident_date": d,
            "incident_type": itype,
            "severity_1to5": severity,
            "property_loss_usd": round(loss, 2)
        })
        incident_id += 1

incidents = pd.DataFrame(inc_rows)

# --- Save outputs
os.makedirs("data", exist_ok=True)
sites.to_csv("data/sites.csv", index=False)
drills.drop(columns=["month"], errors="ignore").to_csv("data/drills.csv", index=False)
maintenance.drop(columns=["month"], errors="ignore").to_csv("data/maintenance.csv", index=False)
incidents.to_csv("data/incidents.csv", index=False)

# Zip for single download
zip_buf = io.BytesIO()
with zipfile.ZipFile(zip_buf, 'w', zipfile.ZIP_DEFLATED) as zf:
    for f in ["data/sites.csv","data/drills.csv","data/maintenance.csv","data/incidents.csv"]:
        zf.write(f)
zip_bytes = zip_buf.getvalue()

print("✔ Generated CSVs in /content/data and a ZIP for download.")
print(sites.shape, drills.shape, maintenance.shape, incidents.shape)

# --- Quick EDA signals (headlines you can talk through)
def pct(x): return f"{100*x:.1f}%"
signals = {}

signals["sites_by_service"] = sites["service_level"].value_counts(normalize=True).round(3).to_dict()
signals["sprinkler_coverage"] = sites["has_sprinklers"].mean()
signals["avg_risk"] = float(sites["risk_score"].mean())
signals["false_alarm_share"] = float((incidents["incident_type"]=="False Alarm").mean())
signals["median_loss_real"] = float(incidents.loc[incidents["incident_type"]=="Real Event","property_loss_usd"].median())

# --- Hypothesis tests (easy, interview-friendly)
out = []

# H1: Sites with high maintenance completion have fewer REAL incidents per site-month.
site_month = site_feats.merge(
    incidents.assign(is_real=(incidents["incident_type"]=="Real Event").astype(int))
    .groupby(["site_id", pd.to_datetime(incidents["incident_date"]).dt.to_period("M").astype(str)])
    .agg(real_incidents=("is_real","sum")).reset_index()
    .rename(columns={"incident_date":"month"}),
    on=["site_id","month"], how="left"
).fillna({"real_incidents":0})

hi_maint = site_month["maint_complete_rate"] >= site_month["maint_complete_rate"].median()
tstat, pval = stats.ttest_ind(site_month.loc[hi_maint,"real_incidents"],
                              site_month.loc[~hi_maint,"real_incidents"],
                              equal_var=False)
out.append(("H1 t-test: real incidents (hi maint vs low)", tstat, pval))

# H2: Drill pass rate relates to severity (Mann-Whitney on severity where incidents occurred)
inc_m = (incidents.assign(month=pd.to_datetime(incidents["incident_date"]).dt.to_period("M").astype(str))
         .merge(drill_agg[["site_id","month","drill_pass_rate"]], on=["site_id","month"], how="left")
         .fillna({"drill_pass_rate":0.0}))
hi_pass = inc_m["drill_pass_rate"] >= inc_m["drill_pass_rate"].median()
u, p = stats.mannwhitneyu(inc_m.loc[hi_pass,"severity_1to5"],
                          inc_m.loc[~hi_pass,"severity_1to5"], alternative="two-sided")
out.append(("H2 Mann-Whitney: severity (hi drill pass vs low)", u, p))

# H3: Proportion test — pass rate Gold vs Bronze service levels
gold = drills.merge(sites[["site_id","service_level"]], on="site_id")
p_gold = gold.loc[gold["service_level"]=="Gold","passed"]
p_bronze = gold.loc[gold["service_level"]=="Bronze","passed"]
# two-proportion z-test
def two_prop_z(x1, n1, x2, n2):
    p_pool = (x1+x2)/(n1+n2)
    se = np.sqrt(p_pool*(1-p_pool)*(1/n1+1/n2))
    z = (x1/n1 - x2/n2)/se
    p = 2*(1-stats.norm.cdf(abs(z)))
    return z, p
z, pz = two_prop_z(p_gold.sum(), p_gold.size, p_bronze.sum(), p_bronze.size)
out.append(("H3 2-prop z: drill pass rate (Gold vs Bronze)", z, pz))

# H4: Chi-square — contingency of incident_type vs sprinkler presence (at site level)
inc_sites = incidents.merge(sites[["site_id","has_sprinklers"]], on="site_id")
tab = pd.crosstab(inc_sites["incident_type"], inc_sites["has_sprinklers"])
chi2, pchi, _, _ = stats.chi2_contingency(tab)
out.append(("H4 Chi-square: incident_type ~ sprinklers", chi2, pchi))

# H5: Simple logistic regression — Real Event (1) ~ risk_score + maint_complete_rate + drill_complete_rate + sprinklers
df_lr = inc_m.merge(site_feats[["site_id","month","risk_score","maint_complete_rate","drill_complete_rate","has_sprinklers"]],
                    on=["site_id","month"], how="left")
df_lr["real"] = (df_lr["incident_type"]=="Real Event").astype(int)
df_lr = df_lr.dropna()
model = smf.logit("real ~ risk_score + maint_complete_rate + drill_complete_rate + has_sprinklers", data=df_lr).fit(disp=False)
summary_text = model.summary2().tables[1].to_string()

print("\n=== Quick EDA Signals ===")
print(f"Service mix: {signals['sites_by_service']}")
print(f"Sprinkler coverage: {pct(signals['sprinkler_coverage'])}")
print(f"Avg risk score: {signals['avg_risk']:.2f}")
print(f"False alarm share: {pct(signals['false_alarm_share'])}")
print(f"Median loss (real events): ${signals['median_loss_real']:,.0f}")

print("\n=== Hypothesis Tests ===")
for name, stat, p in out:
    print(f"{name}: stat={stat:.3f}, p={p:.4g}")
print("\n=== Logistic Regression (Real Event=1) ===")
print(summary_text)

# --- Export a talk-track cheat sheet
with open("data/talk_track.txt","w") as f:
    f.write("SILCO Synthetic Model — Talk Track (quick bullets)\n")
    f.write("--------------------------------------------------\n")
    f.write(f"- Sites: {len(sites):,} | Drills: {len(drills):,} | Maint: {len(maintenance):,} | Incidents: {len(incidents):,}\n")
    f.write(f"- Sprinkler coverage: {pct(signals['sprinkler_coverage'])}; Avg risk score: {signals['avg_risk']:.2f}\n")
    f.write(f"- Incident mix: False alarms {pct(signals['false_alarm_share'])}, median loss on real events ${signals['median_loss_real']:,.0f}\n\n")
    f.write("Hypotheses (directional):\n")
    for name, stat, p in out:
        f.write(f"  * {name} — p={p:.4g}\n")
    f.write("\nInterpretation framing:\n")
    f.write("- Higher maintenance & drill completion are associated with fewer/milder real events (directionally consistent).\n")
    f.write("- Sprinklers shift incident mix toward fewer real events and lower losses.\n")
    f.write("- Service level correlates with readiness (Gold > Bronze drill pass rates).\n")
    f.write("\nNext-step value ideas:\n")
    f.write("- Targeted outreach: sites with low maint/drill completion and high risk.\n")
    f.write("- False alarm reduction program for specific markets.\n")
    f.write("- Budgeting: tie maintenance spend to risk-adjusted incident reduction.\n")

print("\n✔ Wrote data/talk_track.txt (phone-ready bullets).")
print("To download the ZIP in Colab: from the left Files pane, right-click -> Download on /content/data or create a download link below.")

# Optional: create a base64 link (commented out in Colab)
# from google.colab import files
# files.download("data/sites.csv")
# files.download("data/drills.csv")
# files.download("data/maintenance.csv")
# files.download("data/incidents.csv")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.3 MB/s eta 0:00:00
✔ Generated CSVs in /content/data and a ZIP for download.
(3500, 13) (102759, 7) (42495, 8) (21980, 6)


/usr/local/lib/python3.12/dist-packages/scipy/_lib/deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)
/tmp/ipython-input-3619883943.py:266: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  u, p = stats.mannwhitneyu(inc_m.loc[hi_pass,"severity_1to5"],



=== Quick EDA Signals ===
Service mix: {np.str_('Silver'): 0.446, np.str_('Gold'): 0.356, np.str_('Bronze'): 0.198}
Sprinkler coverage: 71.4%
Avg risk score: 0.38
False alarm share: 51.6%
Median loss (real events): $29,464

=== Hypothesis Tests ===
H1 t-test: real incidents (hi maint vs low): stat=nan, p=nan
H2 Mann-Whitney: severity (hi drill pass vs low): stat=nan, p=nan
H3 2-prop z: drill pass rate (Gold vs Bronze): stat=38.284, p=0
H4 Chi-square: incident_type ~ sprinklers: stat=0.429, p=0.5123

=== Logistic Regression (Real Event=1) ===
                           Coef.  Std.Err.          z         P>|z|    [0.025    0.975]
Intercept              -0.216049  0.065791  -3.283867  1.023931e-03 -0.344998 -0.087101
has_sprinklers[T.True] -0.000174  0.036160  -0.004815  9.961583e-01 -0.071046  0.070698
risk_score              0.067800  0.111784   0.606526  5.441653e-01 -0.151292  0.286892
maint_complete_rate    -0.063344  0.036934  -1.715069  8.633253e-02 -0.135732  0.009045
drill_compl